# CoursewareHubのセットアップ--ローカルユーザ認証

---

VCノードにCoursewareHub環境を構築します。

## 概要

このNotebookで構築するCoursewareHubの構成要素を以下に示します。

![構成](images/cw-121-01.png)

各コンテナの役割や処理の詳細については https://github.com/NII-cloud-operation/CoursewareHub-LC_jupyterhub-deploy を参照してください。

> このNotebookで構築するCoursewareHubでは学認との連携を行いません。ローカルユーザとして登録されているユーザからのみの利用を想定しています。

### 事前に準備が必要となるものについて

このNotebookを実行するにあたって、あらかじめ準備が必要となるものについて以下に記します。

* CoursewareHubのサーバ証明書
    - CoursewareHubではHTTPSによる公開を行うためサーバ証明書とその秘密鍵を準備する必要があります。また必要に応じてサーバ証明書の中間CA証明書を準備してください。

### UnitGroup名

CoursewareHubの構築環境となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

上のセルの出力結果を参考にして、UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

In [ ]:
target_nodes = f'{ugroup_name}_worker'

!ansible {target_nodes} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

## パラメータの設定

### Single-user Server

ユーザの利用環境となるSingle-user Serverに関するパラメータを指定します。

#### ユーザインターフェース

Single-user Serverのデフォルト表示をClassic Notebook InterfaceとJupyterLabのどちらにするかを指定します。

ユーザインターフェースの選択はデフォルトのURLにより指定します。次のセルで指定してください。指定しない場合は JupyterLab がデフォルトの表示となります

In [ ]:
# (例)
# jupyterhub_singleuser_default_url = '/tree'   # Classic Notebook Interface
# jupyterhub_singleuser_default_url = '/lab'     # JupyterLab

jupyterhub_singleuser_default_url = '/tree' 

#### リソース制限の設定

Single-user Serverに割り当てるリソースの制限に関する設定を行います。

##### リソース制限の設定方法

CoursewareHubではユーザの役割に応じて利用方法が異なることがあります。例えば講師権限ユーザーは採点を行う場合などに複数のNotebookを同時に開くことがあります。そのような利用を行う場合、一般の受講者ユーザよりも多くのリソースが必要となります。また講師権限を与えないが、採点の補助を行う役割をもつTA(Teaching Assistant)のユーザーがいる場合があります。それらのユーザには一般の受講者ユーザーとは異なるリソース制限設定を行うことが考えられます。

これらのことに対応するためにCoursewareHubでは権限もしくは所属グループに応じたリソース制限の設定を行うことができます。

リソース制限の設定はYAMLファイルで行います。YAMLファイルの記述例を以下に示します。

```yaml
groups:
    student:
        mem_limit: 1G
        cpu_limit: 1.0
        priority: 10
    teaching-assistant:
        mem_limit: 2G
        cpu_limit: 2.0
        mem_guarantee: 1G
        cpu_guarantee: 0.5
        priority: 1
admin:
    mem_limit: 5G
default:
    mem_limit: 1G
    cpu_limit: 2.0
    mem_guarantee: 1G
    cpu_guarantee: 0.5
```

上の例では `student`グループ、`teaching-assistant`グループ、講師権限ユーザ(`admin`)、それ以外のユーザ（デフォルト設定）についてリソース制限の設定を行っています。複数のグループに所属するユーザについては、グループの`priority`が小さいほうのグループの設定が優先されます。上記の例では`student`グループ、`teaching-assistant`グループの両方に属するユーザは `priority`が`1`となっている`teaching-assistant`グループの設定が優先されます。

コンテナに対するリソース制限設定として以下の４つの属性を指定することができます。

* [mem_guarantee](https://jupyterhub.readthedocs.io/en/stable/api/spawner.html#jupyterhub.spawner.Spawner.mem_guarantee)
    - コンテナの使用が保証されるメモリサイズの下限
* [mem_limit](https://jupyterhub.readthedocs.io/en/stable/api/spawner.html#jupyterhub.spawner.Spawner.mem_limit)
    - コンテナが使用可能なメモリのサイズの上限
* [cpu_guarantee](https://jupyterhub.readthedocs.io/en/stable/api/spawner.html#jupyterhub.spawner.LocalProcessSpawner.cpu_guarantee)
    - コンテナの使用が保証される CPU 使用率の下限
* [cpu_limit](https://jupyterhub.readthedocs.io/en/stable/api/spawner.html#jupyterhub.spawner.LocalProcessSpawner.cpu_limit)
    - コンテナが使用可能な CPU 使用率の上限

##### リソース制限の設定を行うYAMLファイルを作成する

前節で説明したYAMLファイルを作成します。

リソースファイルを作成するディレクトリを作成します。

In [ ]:
from tempfile import mkdtemp
from pathlib import Path
rsc_pdir = Path('./rsc').absolute()
rsc_pdir.mkdir(exist_ok=True)
rsc_dir = Path(mkdtemp(dir=rsc_pdir))

YAMLファイルを作成します。

リソース制限の設定に対応する内容に次のセルを変更してください。その後、セルを実行するとYAMLファイルが作成されます。

In [ ]:
%%writefile {rsc_dir}/resource.yaml
default:
    mem_limit: 1G
    mem_guarantee: 1G
    cpu_limit: 1.0

admin:
    mem_limit: 4G
    mem_guarantee: 1G
    cpu_limit: 1.0

groups:
    staff:
        mem_limit: 4G
        mem_guarantee: 1G
        cpu_limit: 1.0
        priority: 1


YAMLファイルの記述内容が妥当であるかをチェックします。

次のセルが正常に実行できることを確認してください。実行結果がエラーとなった場合はYAMLファイルの記述内容に問題があります。上のセルを unfreeze して記述内容を修正して再実行してください。

In [ ]:
import jsonschema
import json
import yaml

!ansible -c local {target_hub} -m get_url -a \
    'url=https://raw.githubusercontent.com/NII-cloud-operation/CoursewareHub-LC_platform/master/jupyterhub/resources-schema.json \
    dest={rsc_dir}/resources-schema.json'
with (rsc_dir / 'resources-schema.json').open() as f:
    resources_config_schema = json.load(f)

with (rsc_dir / 'resource.yaml').open() as f:
    resources_config = yaml.load(f, Loader=yaml.SafeLoader)
    jsonschema.validate(resources_config, resources_config_schema)

print(json.dumps(resources_config, indent=2))

### JupyterHub

JupyterHubコンテナに設定するパラメータを指定します。

#### Cullingの設定

アイドル状態のsingle-userサーバーを自動的に停止する機能に関するパラメータを指定します。

ここで指定できるパラメータを以下に示します。

|項目 | 説明 | デフォルト値|
|:---|:---|--:|
|cull_server              | アイドル状態のNotebookサーバーの停止を有効／無効化 (yes/no/0/1)| no |
|cull_server_idle_timeout | アイドル状態のNotebookサーバーの停止までのタイムアウト時間(秒)     | 600 |
|cull_server_max_age      | アイドル状態でなくてもNotebookサーバーを停止するまでの時間(秒)     | 0 |
|cull_server_every       | Notebookのアイドル状態のタイムアウトのチェック間隔(秒)             | 0 |



各パラメータの値を以下のセルで指定してください。パラメータの設定が不要な場合は、そのセルを実行せずにスキップしてください。

In [ ]:
# (例)
# cull_server = "yes"       # 自動停止を有効にする
# cull_server = "no"        # 自動停止を無効にする

cull_server =

In [ ]:
# (例)
# cull_server_idle_timeout = 10 * 60               # １０分間
# cull_server_idle_timeout = 12 * 60 * 60      # 12時間

cull_server_idle_timeout = 

In [ ]:
# (例)
# cull_server_max_age = 0
# cull_server_max_age = 24 * 60 * 60      # 24時間

cull_server_max_age = 

In [ ]:
# (例)
# cull_server_every = 0
# cull_server_every = 60

cull_server_every = 

#### JupyterHubのSpawnerの設定

single-userサーバーの起動に関するパラメータを指定します。

ここで指定できるパラメータを以下に示します。

|項目 | 説明 | デフォルト値 |
|:---|:---|---:|
|concurrent_spawn_limit   | ユーザーコンテナ同時起動処理数                                 | 20 |
|spawner_http_timeout     | ユーザーNotebookサービス 起動タイムアウト時間(秒)              | 120 |
|spawner_start_timeout | アイドル状態のNotebookサーバーの停止までのタイムアウト時間(秒)  | 300 |

各パラメータの値を以下のセルで指定してください。パラメータの設定が不要な場合は、そのセルを実行せずにスキップしてください。

In [ ]:
# (例)
# concurrent_spawn_limit = 20

concurrent_spawn_limit = 

In [ ]:
# (例)
# spawner_http_timeout = 120

spawner_http_timeout = 

In [ ]:
# (例)
# spawner_start_timeout = 300

spawner_start_timeout = 

#### ロゴ

JupyterHubのロゴをカスタマイズしたい場合は、このNotebookを実行している環境にロゴファイルを配置してそのパスを次のセルで指定してください。指定しない場合はCoursewareHubのデフォルトのロゴが表示されます。

In [ ]:
# (例)
# logo_file = 'jupyterhub-logo.png'

#### カスタマイズ

JupyterHubの設定ファイルを追加することでJupyterHubコンテナの動作をカスタマイズすることがすることができます。

例えば共有ディレクトリ機能を使う場合は、以下のような設定ファイルをこのNotebookを実行している環境に用意します。

```python
from docker.types import Mount

extra_user_mounts = []
extra_user_mounts.append(
    Mount(
        type="bind",
        target="{homedir}/share",
        source="/share",
        read_only=False,
    )
)
c.CoursewareUserSpawner.extra_user_mounts = extra_user_mounts
```

そして、用意した設定ファイルの名前を次のセルのリストに追加してください。ファイル名の末尾は`.py`とする必要があります（ただし共有ディレクトリ機能を利用するには、この他にNFSの設定を行う必要があります）。

In [ ]:
jupyterhub_config_list = [
    # (例)
    #  'share_jupyterhub_config.py',
]

設定ファイルの記述方法については以下のページなどを参照してください。

* [JupyterHub - Configuration Reference](https://jupyterhub.readthedocs.io/en/stable/reference/config-reference.html)

### データベース

CoursewareHubのデータを保存するデータベース(PostgreSQL)に関するパラメータを指定します。

CoursewareHubが利用するデータベース名を指定してください。

In [ ]:
# (例)
# db_name = 'jupyterhub'

db_name = 'jupyterhub'

データベースのユーザ名を指定してください。

In [ ]:
# (例)
# db_user = 'jhauth'

db_user = 'jhauth'

データベースに接続する際のパスワードを指定してください。

In [ ]:
from getpass import getpass
db_pass = getpass()

### auth-proxy

auth-proxyコンテナに関するパラメータを設定します。

SimpleSAMLphp cronモジュールのトリガー用secret文字列を生成します。

In [ ]:
import random
import string
cron_secret = ''.join([random.choice("abcdef" + string.digits) for _ in range(32)])
cron_secret

### registry

registryコンテナに関するパラメータを設定します。

レジストリのパスワードを生成します。

In [ ]:
registry_pass = ''.join([random.choice("abcdef" + string.digits) for _ in range(32)])
registry_pass

### オーバーレイネットワークの指定

CoursewareHubのコンテナはDocker Swarmのオーバーレイネットワークで実行されます。オーバーレイネットワークに割り当てるサブネットを指定します。

> 基本的にはデフォルト値からの変更は不要ですが、VCノードに割り当てられているIPアドレスと範囲が重複している場合は他のサブネットに変更して下さい。

In [ ]:
# (例)
# cousewarehub_backend = '10.100.0.0/20'

cousewarehub_backend = '10.100.0.0/20'

指定されたサブネットがVCノードに割り当てられているIPアドレスと重なっていないことをチェックします。次のセルを実行してエラーとならないことを確認してください。

In [ ]:
!ansible-playbook -v -e cousewarehub_backend={cousewarehub_backend} -l {ugroup_name} \
    playbooks/check-subnet.yml

### 管理者情報の設定

管理者として登録するメールアドレスを指定します。CoursewareHubに管理者としてログインするにはここで指定したメールアドレスをログイン画面で入力します。

In [ ]:
# (例)
# teacher_email = 'admin@example.org'

teacher_email = 

管理者のパスワードを入力してください。

In [ ]:
from getpass import getpass
teacher_password = getpass()

### パラメータの保存

ここまで指定したパラメータを ansible の変数として `group_vars`ファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

teacher_email_list = gvars.get('teacher_email_list', [])
teacher_email_list.append(teacher_email)
teacher_email_list = list(set(teacher_email_list))

gvars.update({
    'rsc_yml': f'{rsc_dir}/resource.yaml',
    'db_name': db_name,
    'db_user': db_user,
    'db_pass': db_pass,
    'cousewarehub_backend': cousewarehub_backend,
    'cron_secret': cron_secret,
    'registry_pass': registry_pass,
    'teacher_email_list': teacher_email_list,
})

if 'logo_file' in vars():
    logo_file = str(Path(logo_file).resolve())
if 'jupyterhub_config_list' in vars():
    jupyterhub_config_list = [str(Path(x).resolve()) for x in jupyterhub_config_list]

jupyterhub_param_names = [
    'cull_server',
    'cull_server_idle_timeout',
    'cull_server_max_age',
    'cull_server_every',
    'concurrent_spawn_limit',
    'spawner_http_timeout',
    'spawner_start_timeout',
    'logo_file',
    'jupyterhub_config_list',
    'intermediate_certfile',
    'jupyterhub_singleuser_default_url',
]

for name in jupyterhub_param_names:
    if name in vars():
        gvars[name] = vars()[name]
    elif 'name' in gvars:
        del(gvars['name'])

gvars_path.rename(gvars_path.parent / f".{gvars_path.stem}.old")
with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)
    
!cat group_vars/{ugroup_name}

## 証明書の配置

auth-proxyコンテナで使用するサーバ証明書、秘密鍵などのファイルを配置します。出どころなどの情報を必要以上に残さないためにNotebookからの操作ではなく、ターミナルなどから **managerノードに ssh でログインして操作を行ってください**。

manager ノードのIPアドレスを確認します。表示されたIPアドレスに対して、ユーザ名`vcp`と「VCノード作成」のNotebookで設定したSSHの秘密鍵を指定することで manager ノードにsshでログインできます。

In [ ]:
!ansible {target_hub} -c local -m debug -a 'var=servicenet_ip'

サーバ証明書、秘密鍵をそれぞれ以下のセルの実行結果に表示されるパスに配置してください。

In [ ]:
!ansible {target_hub} -c local -m debug -a 'msg="/home/{{{{ansible_user}}}}/certs/{{{{master_fqdn}}}}.cer"'

In [ ]:
!ansible {target_hub} -c local -m debug -a 'msg="/home/{{{{ansible_user}}}}/certs/{{{{master_fqdn}}}}.key"'

中間CA証明書がある場合は、サーバ証明書と同じディレクトリに配置してください。中間CA証明書を配置した場合は次のセルにそのファイル名を指定してください。

In [ ]:
# (例)
# intermediate_certfile = 'nii-odca4g7rsa.cer'

intermediate_certfile = 

また個別の中間CA証明書を配置するのではなく、中間CA証明書を連結したファイルを配置することもできます。その場合は次のセルの実行結果に表示されるパスに配置してください。中間CA証明書を連結したファイルが存在している場合は、個別の中間CA証明書を指定しても**連結したファイルの方が優先**されます。

In [ ]:
!ansible {target_hub} -c local -m debug -a 'msg="/home/{{{{ansible_user}}}}/certs/{{{{master_fqdn}}}}.chained.cer"'

**証明書などの配置を行った後に、これ以降の操作を行ってください。**

証明書が配置されたことを確認します。managerノードに配置されたサーバ証明書の内容を表示してみます。

In [ ]:
!ansible {target_hub} -a \
    'openssl x509 -noout -text -in certs/{{{{master_fqdn}}}}.cer'

秘密鍵の内容を表示してみます。

In [ ]:
!ansible {target_hub} -m shell -a \
    'openssl ec -noout -text -in  certs/{{{{master_fqdn}}}}.key || \
    openssl rsa -noout -text -in  certs/{{{{master_fqdn}}}}.key'

中間CA証明書を指定した場合は、そのファイルがmanagerノードに配置されていることを確認します。また、中間CA証明書のパスをgroup_varsに保存します。

In [ ]:
if "intermediate_certfile" in vars():
    !ansible {target_hub} -a \
        'openssl x509 -noout -text -in certs/{intermediate_certfile}'
    import yaml
    from pathlib import Path
    gvars_path = Path(f"group_vars/{ugroup_name}")
    with gvars_path.open() as f:
        gvars = yaml.safe_load(f)
    gvars["intermediate_certfile"] = intermediate_certfile
    with gvars_path.open(mode='w') as f:
        yaml.safe_dump(gvars, stream=f)

中間CA証明書を連結したファイルを配置した場合は、その内容を確認します。

In [ ]:
!ansible {target_hub} -m shell -a 'chdir=certs \
    test ! -f {{{{master_fqdn}}}}.chained.cer || \
    ( openssl crl2pkcs7 -nocrl -certfile {{{{master_fqdn}}}}.chained.cer | openssl pkcs7 -print_certs -noout )'

## CoursewareHubのセットアップ

CoursewareHubを構成するサービスやコンテナのセットアップを行います。

### restuserのインストール

JupyterHubコンテナからホスト環境のローカルユーザ情報を取得するために利用する[restuser](https://github.com/minrk/restuser)をインストールします。

 restuserはGitHubに公開されている
[NII-cloud-operation/CoursewareHub-LC_jupyterhub-deploy](https://github.com/NII-cloud-operation/CoursewareHub-LC_jupyterhub-deploy)の中にあるansibleのタスクを利用してインストールします。

まずGitHubから取得したファイルを格納する作業ディレクトリを作成します。

In [ ]:
import tempfile
work_dir = tempfile.mkdtemp()

GitHubからソースツリーを取得します。

In [ ]:
!git clone https://github.com/NII-cloud-operation/CoursewareHub-LC_jupyterhub-deploy.git {work_dir}/jupyterhub-deploy

直近のコミットログを確認します。

In [ ]:
!cd {work_dir}/jupyterhub-deploy && git log -n 5

restuserに関するroleをコピーします。

In [ ]:
!cp -a {work_dir}/jupyterhub-deploy/roles/restuser playbooks/roles/

取得したroleを利用してrestuserのインストールを行います。まず、実際の変更を行う前にドライラン（チェックモード）でPlaybookを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/install-restuser.yml || true

実際にrestuserのインストールを行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/install-restuser.yml

インストールされたファイルを確認します。

In [ ]:
!ansible {target_hub} -a 'tree /srv/restuser'

restuserはベースコンテナ環境のサービスとしてインストールされています。restuserサービスの状態を確認します。次のセルの出力結果に`Active: active (running)` と表示されることを確認してください。

In [ ]:
!ansible {target_hub} -b -a 'systemctl status restuser'

`restuser`によってユーザ情報が取得できることを確認します。HTTPの応答がOK(200)となり、以下の情報が取得できることを確認してください。

```
{"name": "vcp", "dir": "/home/vcp", "shell": "/bin/bash", "uid": 1000, "gid": 1000}
```

In [ ]:
!ansible {target_hub} -b \
    -a 'curl -sf --unix-socket /var/run/restuser.sock -X POST localhost/{{{{ansible_user}}}}'

作業ディレクトリを削除します。

In [ ]:
!rm -rf {work_dir}

### JupyterHubコンテナに関するファイルの配置

JupyterHubコンテナを実行するために必要となるファイルを配置します。

ここで実行するplaybookが配置するファイル、ディレクトリを以下の表に示します。

|パス|用途|
|:---|:---|
|/var/jupyterhub/resource.yaml|リソース割り当て設定ファイル|
|/var/jupyterhub/logo.png|ロゴファイル|
|/etc/jupyterhub/jupyterhub_config.d/|カスタマイズ設定ファイルディレクトリ|


まず、実際に設定を変更する前にドライラン（チェックモード）でansibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-jupyterhub.yml || true

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/setup-jupyterhub.yml

配置したファイルを確認します。

In [ ]:
!ansible {target_hub} -a 'tree -F {{{{jupyterhub_cfg_dir}}}}'

In [ ]:
!ansible {target_hub} -a 'tree -F {{{{jupyterhub_var_dir}}}}'

### PostgreSQLコンテナに関するファイルの配置

PostgreSQLコンテナに関するファイル、ディレクトリのセットアップを行います。

ここで実行するplaybookが配置するファイル、ディレクトリを以下の表に示します。

|パス|用途|
|:---|:---|
|/jupyter/psql/data/|データディレクトリ|
|/jupyter/psql/create.sql|初期実行SQLファイル|


まず、実際に設定を変更する前にドライラン（チェックモード）でansibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-postgres.yml

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/setup-postgres.yml

配置したファイルを確認します。

In [ ]:
!ansible {target_hub} -b -a 'tree -F {{{{postgres_dir}}}}'

### auth-proxyコンテナに関するファイルの配置

auth-proxyコンテナを実行するために必要となるファイルを配置します。

ここで実行するplaybookが配置するファイル、ディレクトリを以下の表に示します。

|パス|用途|
|:---|:---|
|/etc/jupyterhub/hub-const.php|auth-proxyコンテナの設定ファイル|
|/etc/jupyterhub/nginx/certs/auth-proxy.chained.cer|nginxで利用するサーバ証明書|
|/etc/jupyterhub/nginx/certs/auth-proxy.key|nginxで利用する秘密鍵|
|/etc/jupyterhub/simplesamlphp/cert/auth-proxy.cer|simplesamlphpで利用する証明書|
|/etc/jupyterhub/simplesamlphp/cert/auth-proxy.key|simplesamlphpで利用する秘密鍵|
|/etc/jupyterhub/simplesamlphp/metadata/xml/|学認連携を行う場合にメタデータを配置するディレクトリ|
|/etc/jupyterhub/lti.json|LTI連携に関する設定ファイル|
|/etc/jupyterhub/lti/private.key|LTI連携で利用する秘密鍵|


まず、実際に設定を変更する前にドライラン（チェックモード）でansibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-auth-proxy.yml || true

実際に設定変更を行います。

In [ ]:
!ansible-playbook -v -l {target_hub} playbooks/setup-auth-proxy.yml

配置したファイルを確認します。JupyterHubコンテナのために配置したファイル、ディレクトリも含まれています。

In [ ]:
!ansible {target_hub} -a 'tree -F {{{{jupyterhub_cfg_dir}}}}'

### registryコンテナに関するファイルの配置

registryコンテナを実行するために必要となるファイルを配置します。

ここで実行するplaybookが配置するファイル、ディレクトリを以下の表に示します。

|パス|用途|
|:---|:---|
|/jupyter/registry/data/|データディレクトリ|
|/jupyter/registry/htpasswd|認証に関する設定ファイル|


まず、実際に設定を変更する前にドライラン（チェックモード）でansibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-registry.yml

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} -v playbooks/setup-registry.yml

配置したファイルを確認します。

In [ ]:
!ansible {target_hub} -b -a 'tree -F -L 1 {{{{registry_dir}}}}'

## コンテナの起動

### docker-compose.yml の配置

複数のコンテナに関する設定をまとめて扱うために `docker-compose.yml` を利用します。ここまでに指定されたパラメータに応じた`docker-compose.yml`を生成しmanagerノードの`/opt/coursewarehub/docker-compose.yml`に配置します。

まずはチェックモードで確認を行います。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/deploy-docker-compose.yml || true

実際に`docker-compose.yml`の配置を行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/deploy-docker-compose.yml

配置したファイルを確認します。異なるパラメータで何度か配置を行った場合は、以前に配置したファイルがバックアップファイルとして残されています。

In [ ]:
!ansible {target_hub} -b -a 'tree -F {{{{compose_dir}}}}'

### コンテナの起動

コンテナを起動します。

必要なコンテナイメージを事前に取得しておきます。

In [ ]:
!ansible {target_hub} -a 'chdir={{{{compose_dir}}}} \
    docker compose pull -q'

実際にコンテナの起動を行います。

In [ ]:
!ansible {target_hub} -a 'chdir={{{{compose_dir}}}} \
    docker stack deploy -c docker-compose.yml coursewarehub'

コンテナの起動状態を確認します。

In [ ]:
!ansible {target_hub} -a 'docker service ls'

全てのコンテナが起動するまで待ち合わせを行います。

> 次のセルの実行結果がエラーとなる場合は、その後のセルを実行してコンテナの実行状況やログを確認してください。

In [ ]:
import time
import sys


cmd = "docker service ls --format '{{.Name}} {{.Replicas}}' | grep '1/1' | grep coursewarehub_ | wc -l"
cmd = "docker stack services --format '{{.Replicas}}' coursewarehub | grep '1/1' | wc -l"
for retry in range(18):
    time.sleep(10)
    try:
        !ansible {target_hub} -m shell -a "{{% raw %}} [ \$({cmd}) -eq 4 ] {{% endraw %}}"
        break
    except:
        print('retry', file=sys.stderr)
else:
    !ansible {target_hub} -a 'docker service ls'
    raise RuntimeError("起動処理が完了しませんでした。")

`postgres` コンテナのログを表示してみます。

In [ ]:
!ansible {target_hub} -a 'docker service logs coursewarehub_postgres'

`jupyterhub` コンテナのログを表示してみます。

> PostgreSQLに接続できないなどのエラーが表示されていないことを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker service logs coursewarehub_jupyterhub'

`auth-proxy` コンテナのログを表示してみます。

> 証明書設定に誤りがあるなどのエラーが表示されていないことを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker service logs coursewarehub_auth-proxy'

### registryにイメージをpushする

single-userサーバの初期イメージをregistryにpushします。

まずはチェックモードで確認を行います。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/deploy-singleuser-container.yml || true

実際の操作を行います。コンテナイメージの取得などを行うので５分前後の時間がかかります。

In [ ]:
!ansible-playbook -l {target_hub} -v playbooks/deploy-singleuser-container.yml

workerノードに初期イメージを配置します。まずチェックモードで確認を行います。

In [ ]:
!ansible-playbook -l {target_nodes} -CDv playbooks/deploy-singleuser-image.yml || true

実際の操作を行います。

In [ ]:
!ansible-playbook -l {target_nodes} -v playbooks/deploy-singleuser-image.yml

## 管理者の登録

CoursewareHubに管理ユーザを登録します。

ここでセットアップする主な項目を以下に示します。

* 全てのノード(manager, worker)にユーザーアカウントを作成する
* sshの鍵ペアを作成する
* ansibleのインベントリファイルを作成する
* sudo権限を与える
* JupyterHubユーザとして登録する

### パラメータの確認

CoursewareHubではメールアドレスから一定のルールで導き出された名前をローカルユーザ名として利用します。管理ユーザのローカルユーザ名を確認します。

In [ ]:
%run scripts/cwh.py
teacher_id = get_username_from_mail_address(teacher_email)
print(teacher_id)

管理ユーザのハッシュ化されたパスワードを取得します。

In [ ]:
from crypt import crypt

encrypted_teacher_password = crypt(teacher_password)
encrypted_teacher_password

JupyterHubのユーザ登録に必要となるcookie idを作成します。

In [ ]:
import random
import string
cookie_id = ''.join([random.choice("abcdef" + string.digits) for _ in range(32)])
cookie_id

### 管理ユーザの登録

ユーザに関するパラメータをextra_varsとして記述したJSONファイルを作成します。

In [ ]:
import tempfile
from pathlib import Path
import yaml

work_dir = tempfile.mkdtemp()
extra_vars = Path(work_dir) / 'group.yml'
with extra_vars.open(mode='w') as f:
    yaml.safe_dump(dict(
        teacher_id=teacher_id,
        cookie_id=cookie_id,
        encrypted_teacher_password=encrypted_teacher_password,
        teacher_email=teacher_email,
    ), stream=f)

managerノードに管理者ユーザを登録するplaybookを実行します。まずはチェックモードで確認を行います。

In [ ]:
!ansible-playbook -l {target_hub} -CDv -e @{str(extra_vars)} playbooks/setup-admin-user.yml || true

実際の設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} -Dv -e @{str(extra_vars)} playbooks/setup-admin-user.yml

managerノードに作成した管理ユーザと同じアカウントをworkerノードにも作成します。managerノードに作成した管理ユーザのUID, GIDの値を確認します。

In [ ]:
out = !ansible {target_hub} -a 'id -u {teacher_id}' 2> /dev/null
teacher_uid = out[-1]
teacher_uid

In [ ]:
out = !ansible {target_hub} -a 'id -g {teacher_id}' 2> /dev/null
teacher_gid = out[-1]
teacher_gid

workerノードに管理者ユーザを登録するplaybookを実行します。まずはチェックモードで確認を行います。

In [ ]:
!ansible-playbook -l {target_nodes} -CDv \
    -e teacher_id={teacher_id} -e teacher_uid={teacher_uid} -e teacher_gid={teacher_gid} \
    playbooks/setup-worker-admin-user.yml

実際の設定変更を行います。

In [ ]:
!ansible-playbook -l {target_nodes} -Dv \
    -e teacher_id={teacher_id} -e teacher_uid={teacher_uid} -e teacher_gid={teacher_gid} \
    playbooks/setup-worker-admin-user.yml

### 確認

管理ユーザが作成されたことを確認します。

管理ユーザのホームディレクトリの状態を確認します。以下のファイルが作成されていることを確認してください。

|パス|内容|
|:---|:---|
|ansible/inventory|ansibleのインベントリ|
|.ssh/authorized_keys|公開鍵の登録ファイル|
|.ssh/id_rsa|sshの秘密鍵|
|.ssh/id_rsa.pub|sshの公開鍵|
|info/|コンテンツディレクトリ|
|textbook/|コンテンツディレクトリ|

In [ ]:
!ansible {target_hub} -b -a 'tree -aF {{{{home_dir}}}}/{teacher_id}'

JupyterHubのユーザとして登録されていることを確認します。

PostgreSQLコンテナでJupyterHubのユーザを管理している`users`テーブルと、CoursewareHubのローカル認証情報を管理している`local_users`の内容を表示します。それぞれのテーブルで`teacher_id`に対応するレコードが表示されることを確認してください。

In [ ]:
out = !ansible-playbook -l {target_hub} -e teacher_id={teacher_id} \
    playbooks/show-jupyterhub-user.yml
line = [x for x in out if x.find("show_tables.stdout") >= 0][0]
print(json.loads(f'{{{line}}}')['show_tables.stdout'])

作業ディレクトリを削除します。

In [ ]:
!rm -rf {work_dir}

## コンテンツの配備の準備

CoursewareHubのコンテンツを格納するディレクトリを作成し、コンテンツの配置を行うNotebookを管理者のホームディレクトリに配置します。

ここで実行するplaybookが配置するファイル、ディレクトリを以下の表に示します。

|パス|用途|
|:---|:---|
|/jupyter/admin/admin_tools|コンテンツ用ディレクトリ|
|/jupyter/admin/tools|コンテンツ用ディレクトリ|
|/jupyter/admin/textbook|コンテンツ用ディレクトリ|
|/jupyter/admin/info|コンテンツ用ディレクトリ|
|/jupyter/users/{teacher_id}/D06_CoursewareHubコンテンツの配備.ipynb|コンテンツを配備するnotebook|

`{teacher_id}`は管理ユーザ名によって異なる値となります。

「D06_CoursewareHubコンテンツの配備.ipynb」はNII Cloud Operation Teamにより公開されている[CoursewareHub-LC_deploy/coursewarehub/D06_CoursewareHubコンテンツの配備.ipynb](https://github.com/NII-cloud-operation/CoursewareHub-LC_deploy/blob/main/coursewarehub/D06_CoursewareHub%E3%82%B3%E3%83%B3%E3%83%86%E3%83%B3%E3%83%84%E3%81%AE%E9%85%8D%E5%82%99.ipynb)をGitHubより取得して配備します。

実際に設定を変更する前にドライラン（チェックモード）でansibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv -e teacher_id={teacher_id} playbooks/manage-tools.yml

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} -e teacher_id={teacher_id} playbooks/manage-tools.yml

配置したディレクトリ、ファイルを確認します。

In [ ]:
!ansible {target_hub} -b -a 'tree -F {{{{jupyter_root_dir}}}}/admin'

In [ ]:
!ansible {target_hub} -b -a 'tree -F -L 1 {{{{home_dir}}}}/{teacher_id}'

## CoursewareHubにアクセスする

構築したCoursewareHub環境にアクセスして、正しく動作していることを確認してください。

次のセルを実行すると、構築したCoursewareHubのアドレスを表示します。

In [ ]:
print(f'https://{gvars["master_fqdn"]}')

CoursewareHubの構築後の最初のログインでは、コンテナイメージの取得に時間がかかるためsingle-userサーバが起動するまでに5分程度の時間がかかります。

管理ユーザのホームディレクトリにはコンテンツを配備するためのnotebook「D06_CoursewareHubコンテンツの配備.ipynb」が用意されています。管理ユーザでログインした後に実行してください。

JupyterHub 3.1ではRBACの仕様変更により`JUPYTERHUB_API_TOKEN`のトークンでできる操作の範囲が狭められています。そのため「D06_CoursewareHubコンテンツの配備.ipynb」が配置するもののうち、以下のものについてはAPIトークンの設定方法を変更する必要があります。

* 80_AddLocalUsers.ipynb
* 83_AddUsersToGroup.ipynb
* 84_RemoveUsersFromGroup.ipynb
* 85_ShowUserGroups.ipynb

上記のnotebookではJupyterHubのAPIトークンを設定するセルが以下のような記述となっています。

```
import os
apitoken = os.environ['JUPYTERHUB_API_TOKEN']
api_url = os.environ['JUPYTERHUB_API_URL']
```

このようなセルは、例えば「80_AddLocalUsers.ipynb」の場合、「3.JupyterHubユーザーの追加」にあります。

現時点(2023/10/20)では、このままnotebookを実行するとエラーとなってしまいます。管理ユーザがJupyterHubで明示的に発行したAPIトークンを指定するように変更してください。例えば発行したAPIトークンの値が`xxxxxxxx`の場合、セルの内容を以下のように変更して実行してください。

```
import os
apitoken = 'xxxxxxxx'
api_url = os.environ['JUPYTERHUB_API_URL']
```

管理ユーザがJupyterHubのAPIトークンを発行するには、まず[Control Panel]ボタン（下図の赤丸）を選択して管理画面を表示してください。

![control panel](images/cw-731-10.png)

その後、[Token]タブ（下図の赤丸）を選択することで表示される画面の[Request new API token]ボタンからAPIトークンを発行できます。

![token tab](images/cw-121-02.png)